## Background

A canonical inverse problem is to infer the spatially varying parameter field $k(x)$ in the following elliptic PDE
$$
-\nabla\cdot\left[ k \nabla h\right] = f,
$$
where $h(x)$ is the solution of the PDE and $f(x)$ is a source term.   We consider this PDE in a one dimensional (i.e., $x\in\mathbb{R}$) setting with boundary conditions given by
$$
\begin{aligned}
h(0) &= 0\\
\left. \frac{\partial h}{\partial x}\right|_{x=1} &= 0.
\end{aligned}
$$

This equation can be used to describe many different diffusive processes, such as heat conduction, groundwater flow, and contaminant diffusion.    This example interprets the PDE in the context of groundwater flow, meaning that $h(x)$ represents the hydraulic head in a confined aquifer, $f(x)$ represents recharge of water entering the aquifer, and $k(x)$ is the hydraulic conductivity, which describes how difficult it is for water to pass through the aquifer.

The hydraulic head $h(x)$ can be measured at individual points $x_1,x_2,\ldots,x_M$ by drilling boreholes into an aquifer and measuring the pressure in the borehole.  The conductivity $k(x)$ however, cannot be observed directly.   We will therefore consider estimating the hydraulic conductivity $k(x)$ from noisy measurements of $h(x_1), h(x_2),\ldots, h(x_M)$.   

**The goal of this example is to demonstrate the use of MUQ for solving Bayesian inverse with PDE models.**   The model is implemented in a MUQ ModPiece that accepts a discreteized conductivity field $k(x)$ as an input and returns a discretized hydraulic head.  The implementation of the PDE solver, adjoint gradients, and Hessian information is described in detail in the [FlowEquation](../../../../Modeling/FlowEquation/python/DarcyFlow.ipynb) modeling example.  Here we simply use the ModPiece that was implemented in that example.

### Imports

In [1]:
import muq.Modeling as mm 
import muq.SamplingAlgorithms as ms 
import muq.Utilities as mu
import muq.Approximation as ma 
import muq.Optimization as mo

from FlowEquation import FlowEquation

import numpy as np
import matplotlib.pyplot as plt 

## Inverse Problem Formulation

#### Discretization
Assume that $x\in[0,1]$ and that the conductivity field is defined as piecwise constant on $N$ uniformly sized cells dividing up $[0,1]$.  There are $N+1$ nodes in the discretization.


In [2]:
class Discretization:
    
    def __init__(self,numCells):
        self.numCells = numCells
        self.numNodes = numCells+1

        self.nodeLocs = np.linspace(0,1,numCells+1)

        self.cellLocs = 0.5*(self.nodeLocs[:-1] + self.nodeLocs[1:]).reshape(1,-1)

In [3]:
# Define the mesh
numCells = 50
mesh = Discretization(numCells)

#### Prior Distribution

$$
\log k(x) \sim GP(0,k(x,x^\prime))
$$

In [4]:
def CreatePrior(mesh):
    
    # Define the prior distribution
    priorVar    = 1.0
    priorLength = 0.05
    priorNu     = 3.0/2.0
    
    covKernel = ma.MaternKernel(1, priorVar, priorLength, priorNu) # The first argument "1" specifies we are working in 1d
    
    meanFunc = ma.ZeroMean(1,1) # dimension of x, components in k(x) if it was vector-valued

    priorGP = ma.GaussianProcess(meanFunc,covKernel)

    return priorGP.Discretize(mesh.cellLocs)

In [5]:
priorDist = CreatePrior(mesh)
priorDens = priorDist.AsDensity()

#### Likelihood Function
In the finite element solution, the hydraulic conductivity $h(x)$ will then be piecewise linear and can be characterized by values at the $N+1$ nodes in the discretization.   We will assume that 

In [6]:
def GetTrueLogConductivity(mesh):
    return np.cos(10.0*mesh.cellLocs)

In [7]:
def GenerateData(mesh, obsThin, obsVar):

    # Generate the data
    numRefine = 1
    fineMesh = Discretization(numRefine*mesh.numCells)
    trueCond = np.exp( GetTrueLogConductivity(fineMesh) )

    # Create the model with twice the mesh resolution
    recharge = np.ones(fineMesh.numCells)
    mod = FlowEquation(recharge)

    # Solve the forward problem with the true conductivity
    trueSol = mod.Evaluate( trueCond )[0]

    # Take every N node as an "observation"
    slicer = mm.SliceOperator(fineMesh.numNodes,0,fineMesh.numCells,numRefine*obsThin)

    return slicer.Evaluate([trueSol])[0] + np.sqrt(obsVar)*np.random.randn(slicer.outputSizes[0])

In [8]:
def ConstructLikelihood(mesh, data, obsThin, obsVar, forwardMod):
   
    graph = mm.WorkGraph()
    graph.AddNode(mm.ExpOperator(mesh.numCells), "Conductivity")
    
    graph.AddNode(forwardMod, "Forward Model")
    graph.AddEdge("Conductivity", 0, "Forward Model", 0)
    
    graph.AddNode(mm.SliceOperator(mesh.numNodes,0,mesh.numCells,obsThin), "Observables")
    graph.AddEdge("Forward Model", 0, "Observables", 0)
    
    likelihood = mm.Gaussian(data, obsVar*np.ones(data.shape[0]))
    
    graph.AddNode(likelihood.AsDensity(), "Likelihood")
    graph.AddEdge("Observables", 0, "Likelihood", 0)
    
    temp = graph.CreateModPiece("Likelihood")
    
    graph.Visualize("LikelihoodGraph.pdf")
    
    return temp#graph.CreateModPiece("Likelihood")

In [9]:
# Generate synthetic "truth" data
obsThin = 4
obsVar = 0.01*0.01
data = GenerateData(mesh, obsThin, obsVar)
    
# Define the forward model
recharge = np.ones(mesh.numCells)
forwardMod = FlowEquation(recharge)

# Build the likelihood function
likelihood = ConstructLikelihood(mesh, data, obsThin, obsVar, forwardMod)

#### Create Posterior

In [10]:
def ConstructPosterior(prior, likelihood):
   
    graph = mm.WorkGraph()
    graph.AddNode(mm.IdentityOperator(mesh.numCells), "Log Conductivity")
    
    graph.AddNode(prior, "Prior")
    graph.AddEdge("Log Conductivity", 0, "Prior", 0)
    
    graph.AddNode(likelihood, "Likelihood")
    graph.AddEdge("Log Conductivity",0, "Likelihood",0)
    
    graph.AddNode(mm.DensityProduct(2), "Posterior")
    graph.AddEdge("Prior",0,"Posterior",0)
    graph.AddEdge("Likelihood",0,"Posterior",1)

    graph.Visualize("PosteriorGraph.pdf")
    
    return graph.CreateModPiece("Posterior")

In [11]:
posterior = ConstructPosterior(priorDens, likelihood)

## Compute MAP Point

In [12]:
def ComputeMAP(logPost, startPt):
    
    options = {
        'Algorithm' : 'NewtonTrust', # Newton-Steihaug trust region method.  Options also include NLOPT optimizers like LBFGS
        'PrintLevel' : 1,
        'Ftol.AbsoluteTolerance' : 1e-4
    }
    
    # Define the objective function as the negative log posterior 
    objective = mo.ModPieceCostFunction(logPost, -1.0)
    
    # Construct the optimizer and minimize the objective
    solver = mo.Optimizer.Construct(objective, options)
    xopt, fopt = solver.Solve([ startPt ])
    
    return xopt

In [13]:
# Use a random draw from the prior as a starting point for MCMC
startPt = priorDist.GetMean()
print(priorDens.Evaluate([startPt]))
print(likelihood.Evaluate([startPt]))
print(posterior.Evaluate([startPt]))
mapPt = ComputeMAP(posterior, startPt)

[array([-9.98692317])]
[array([-584.27674161])]
[array([-594.26366478])]
Using NewtonTrust optimizer...
  Iteration, TrustRadius,       fval,      ||g||
          0,       1.000,  5.943e+02,  7.145e+02


RuntimeError: Could not add an edge from "Forward Model_Gradient[0,0]_Jacobian[0,0]" to "Conductivity_Gradient[0,0]_Jacobian[0,1]" because the source node "Forward Model_Gradient[0,0]_Jacobian[0,0]" does not exist in the graph.

#### Plot the MAP Point

In [ ]:
trueLogK = GetTrueLogConductivity(mesh).ravel()

fig, axs = plt.subplots(nrows=2,figsize=(12,8))
axs[0].plot(mesh.cellLocs.ravel(), mapPt,label='MAP Point')
axs[0].plot(mesh.cellLocs.ravel(), trueLogK, '--k', label='True $\log k(x)$')
axs[0].plot(mesh.cellLocs.ravel(), startPt, '--g', label='Starting Point for Optimization')
axs[0].legend()

truePred = model.Evaluate([trueLogK])[0]
mapPred = model.Evaluate([mapPt])[0]
startPred = model.Evaluate([startPt])[0]

axs[1].plot(mesh.nodeLocs, mapPred, label='Prediction at MAP')
axs[1].plot(mesh.nodeLocs, truePred,'--k',label='True Prediction')
axs[1].plot(mesh.nodeLocs, startPred, '--g', label='Starting Prediction')
axs[1].plot(mesh.nodeLocs[0:-1:obsThin],data,'+k',markersize=12,label='Observations')
axs[1].legend()

plt.show()

## Sample Posterior with DILI

In [ ]:
def SampleDILI(posterior, startPt, numSamps):
    
    lisOpts = {
        'Method' : "MHKernel",
        "Proposal" : "Prop",
        "Prop.Method" : "MALAProposal",
        "Prop.StepSize" : 0.15
    }
    
    csOpts = {
        "Method" : "MHKernel",
        "Proposal" : "Prop",
        "Prop.Method" : "CrankNicolsonProposal",
        "Prop.Beta" : 0.8,
        "Prop.PriorNode" : "Prior"
    }
    
    opts = {
        "NumSamples" : numSamps,
        "BurnIn" : 0,
        "PrintLevel" : 3,
        "HessianType" : "Exact",
        "Adapt Interval" : 0,
        "Initial Weight" : 100,
        "Prior Node" : "Prior",
        "Likelihood Node" : "Likelihood",
        "LIS Block": "LIS",
        "LIS" : lisOpts,
        "CS Block": "CS",
        "CS" : csOpts
    }
    
    # create a sampling problem
    problem = ms.SamplingProblem(posterior)
    
    # Construct the DILI kernel and MCMC sampler
    kernel = ms.DILIKernel(opts, problem)
    sampler = ms.SingleChainMCMC(opts, [kernel])
    
    return sampler.Run([startPt]) 

In [ ]:
numSamps = 100000
diliSamps = SampleDILI(posterior, mapPt, numSamps)

In [ ]:
diliEss = diliSamps.ESS(method='MultiBatch')
print('DILI:\n  Multivariate ESS: {:5d}'.format(int(diliEss[0])))

### Plot DILI Results

In [ ]:
samps = diliSamps.AsMatrix()

fig, axs = plt.subplots(nrows=2,sharex=True,figsize=(14,10))
axs[0].plot(samps[0,:])
axs[0].set_ylabel('$\log K_0\f$',fontsize=16)
axs[0].set_title('Selected Traces for DILI MCMC',fontsize=18)

axs[1].plot(samps[-1,:])
axs[1].set_ylabel('$\log K_{{ {} }}$'.format(samps.shape[0]-1),fontsize=16)
axs[1].set_xlabel('MCMC Step', fontsize=16)
axs[1].set_xlim([0,samps.shape[1]-1])

plt.subplots_adjust(hspace=0.05)

In [ ]:
plt.figure(figsize=(14,5))

plotInd = int(0.5*samps.shape[0])
plt.acorr(samps[plotInd,:]-np.mean(samps[plotInd:]),maxlags=10000)
plt.xlabel('Lag', fontsize=16)
plt.ylabel('Autocorrelation', fontsize=16)
plt.title('Autocorrelation of $\log K_{{ {} }}$ in DILI chain'.format(plotInd),fontsize=18)
plt.show()

## Sample Posterior with pCN

In [ ]:
def SamplePCN(posterior, startPt, numSamps):
    
    kernOpts = {
        "Method" : "MHKernel",
        "Proposal" : "Prop",
        "Prop.Method" : "CrankNicolsonProposal",
        "Prop.Beta" : 0.1,
        "Prop.PriorNode" : "Prior"
    }
    
    opts = {
        "NumSamples" : numSamps, # number of Monte Carlo samples,
        "BurnIn" : 0,
        "PrintLevel" : 3,
        "KernelList" : "Kernel1",
        "Kernel1" : kernOpts
    }
    
    
    #create a sampling problem
    problem = ms.SamplingProblem(posterior)

    sampler = ms.SingleChainMCMC(opts,problem)

    return sampler.Run([startPt]) # Use a true posterior sample to avoid burnin

In [ ]:
pcnSamps = SamplePCN(posterior, startPt, numSamps)

In [ ]:
pcnEss = pcnSamps.ESS(method='MultiBatch')
print('pCN:\n  Multivariate ESS: {:5d}'.format(int(pcnEss[0])))

### Plot pCN Results

In [ ]:
samps = pcnSamps.AsMatrix()

fig, axs = plt.subplots(nrows=2,sharex=True,figsize=(14,10))
axs[0].plot(samps[0,:])
axs[0].set_ylabel('$\log K_0\f$',fontsize=16)
axs[0].set_title('Selected Traces for pCN MCMC',fontsize=18)

axs[1].plot(samps[-1,:])
axs[1].set_ylabel('$\log K_{{ {} }}$'.format(samps.shape[0]-1),fontsize=16)
axs[1].set_xlabel('MCMC Step', fontsize=16)
axs[1].set_xlim([0,samps.shape[1]-1])

plt.subplots_adjust(hspace=0.05)


In [ ]:
plt.figure(figsize=(14,5))

plt.acorr(samps[plotInd,:]-np.mean(samps[plotInd:]),maxlags=10000)
plt.xlabel('Lag', fontsize=16)
plt.ylabel('Autocorrelation', fontsize=16)
plt.title('Autocorrelation $\log K_{{ {} }}$ in pCN chain'.format(plotInd),fontsize=18)
plt.show()